# 【NLP 自然語言處理】2024 FaceBook社團貼文爬蟲 Part 1

## Selenium 爬蟲

### 01. 瀏覽器設定：允許所有網站通知行為

`profile.default_content_setting_values.notifications`：這是 Chrome 設定中負責控制網站通知行為的部分。
- 1：允許對於網站發出的通知。

- 2：阻擋通知。

![notifications](./image/notification.png)

In [1]:
from selenium import webdriver

chrome_options = webdriver.ChromeOptions()

# 允許所有網站通知行為
chrome_options.add_experimental_option(
    "prefs", 
    {
        "profile.default_content_setting_values.notifications": 1
    }
)

### 02. 啟動 webdriver

In [2]:
driver = webdriver.Chrome(options = chrome_options)

### 03. 設定視窗大小

不同的視窗大小會影響網頁排版樣式，間接影響爬蟲的流程，事先設定好可以避免因排版問題而導致爬蟲錯誤。

In [3]:
driver.set_window_size(800, 800)

### 04. 網址導向

利用 `get` 可將 Webdriver 導向到指定的網站頁面。

In [4]:
login_url = 'https://www.facebook.com/login/device-based/regular/login/?login_attempt=1&next=https%3A%2F%2Fwww.facebook.com'
driver.get(login_url)

### 05. 輸入 Input 

利用 `find_element` 可以抓到網頁的指定的位置：

- 第一個參數：指定 Selector 來定向到目標節點：輸入框、按鈕。例如：By.CSS_SELECTOR （使用 CSS Selector）

- 第二個參數：目標節點的位置。例如：#email_container input （目標節點的 CSS 位置）

可用於輸入搜尋框、登入輸入帳密...等

#### Email 輸入框

由下圖可以發現 Email 輸入框對應的節點是標籤 input，但不是只有 Email 輸入框的標籤是 input。

因此，我可以從標籤 input 的前一層（父級）來去指定，我們抓取前一層的屬性 id。在 CSS Selector 的表示為 `#email_container`。

如果要指定屬性 id 為 `email_container` 底下的標籤 input，則空格加上下一層（子級）的 Selector，所以就會是 `#email_container input`

![Email](./image/login.png)

#### Password 輸入框

Password的概念相同，不同的是，我們先嘗試選擇屬性 class，再加上標籤 input。

如果要使用 CSS Selector 來指定屬性 class，則要加上 `.`，例如： `._55r1`。

![Email](./image/pass.png)

但是，可以注意到在這個例子的屬姓 class 是 `_55r1 _1kbt`，中間有了空格。這裡並不是 classname 可以包含空格，而是這裡存在兩個 classname。（理論上沒有上限，屬性值可以有複數個名稱，會以空格來區隔）。

根據這裡包含的 classname，可以有幾種指定方式：

- 指定其中一個 classname：`._55r1`、`._1kbt`。

- 指定所有 classname：`._55r1._1kbt`（兩個 classname 位在同一個標籤下，不存在從屬關係，因此 CSS Selector 不須空格）。

加上標籤 input 後，就會變成 `._55r1._1kbt input`（`input` 在 `_55r1 _1kbt` 下，所以 CSS Selector 須空格）。

In [5]:
from selenium.webdriver.common.by import By


# 建議新建一個新的帳號，避免被鎖
facebook_mail = 'YOUR_FACEBOOK_MAIL'
facebook_password = 'YOUR_FACEBOOK_PASSWORD'

email_element = driver.find_element(By.CSS_SELECTOR,'#email_container input')
password_element = driver.find_element(By.CSS_SELECTOR,'._55r1._1kbt input')

email_element.send_keys(facebook_mail)
password_element.send_keys(facebook_password)

### 06. 點擊按鈕

#### Login 按鈕

由於 id 屬性在這個網站是唯一的，`loginbutton`。因此 CSS Selector 直接設置為 `#loginbutton`即可。

![Email](./image/btn.png)

In [6]:
login_button = driver.find_element(By.CSS_SELECTOR, '#loginbutton')

login_button.click()

前往指定社團頁面

In [7]:
url = 'https://www.facebook.com/groups/traveler168'

driver.get(url)

### 07. 頁面滾動

由於動態網頁的網頁內容是透過Javascript渲染而成，因此進入網頁後不會一次渲染出所有內容，例如：

- FaceBook 社團貼文
- Instagram 搜尋結果頁
- Google Maps 景點評論

上面提到的例子，都必須透過使用者不斷滾動頁面來載入更多內容，而在爬蟲時，我們就必須模擬像這樣的滾動行為，來取得更多資料。

`execute_script` 可以幫助我們在 Webdriver 上執行 Javascript，如此一來便能夠做出更細緻、進階的操作，包含但不限於：頁面滾動、處理彈出窗口、修改網頁元素。

In [17]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

我們可以寫一個迴圈，讓網站可以連續下滑3次。

註：由於網頁 DOM 節點的載入需要時間，滾動頁面時中間需間隔一段時間，否則在節點載入完成前 Javascript 就會先執行完畢。 

In [18]:
import time

counter = 0
while counter <= 3:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # 等待 0.5 秒（確保網頁節點有載入，可根據需求調整等待時間）
    time.sleep(0.5)
    counter += 1

### 08. 關閉 Webdriver

In [10]:
# 註解避免 Webdriver 關閉，這樣就可以接續往下執行
# driver.close()

# 【NLP 自然語言處理】2024 FaceBook社團貼文爬蟲 Part 2

In [11]:
# !pip3 install selenium pandas

在 Facebook 社團爬蟲的下半章節，將開始實際爬取社團內的貼文內容。爬取的資訊包含：

- 作者

- 讚數

- 時間

- 連結

- 貼文內容

### 解析網頁內容

在 Python當中，我們可以借助 BeautifulSoup 來去解析網頁內容。

延續上一篇文，`driver` 會取得瀏覽器頁面的原始 HTML 內容（就如我們實際在畫面上到的那樣）。而 `driver` 帶有 `page_source` 這個屬性，我們可以利用 `driver.page_source` 來取得當前瀏覽頁面的所有 HTML 標記（這並不等於網頁的原始碼）。

既然我們已經可以抓出網頁的 HTML 標記，為何還需要 BeautifulSoup 呢？

`driver.page_source` 回傳的只是提取出來的字串，要以字串的形式來去檢索貼文內容並不方便。BeautifulSoup 可以解析 HTML，從字串轉換為 BeautifulSoup 物件。如此一來，我們就可以使用 BeautifulSoup 提供的方法來查找網頁元素。

In [19]:
from bs4 import BeautifulSoup

page_content = driver.page_source

# 因為要解析的對象是 HTML，所以需指定 html.parser 解析器
soup = BeautifulSoup(page_content, 'html.parser')

將 HTML 標記轉換為 BeautifulSoup 物件後，我們就可以使用 BeautifulSoup 裡的 `select` 方法。這個方法會根據 CSS Selector 來選取 HTML 的元素。

輸出的結果會變成一個列表，包含了所有符合 CSS Selector 的元素。

如何撰寫 CSS Selector 可參考此篇文章：https://medium.com/@xcswap.john/爬蟲必備的html-css-selectors基礎-0dc9bc399fd6

![soup_select_post](./image/soup_select_post.png)

為了抓到更精確的貼文資訊、我們可以先抓每篇貼文的整體元素，後續再對每篇貼文爬取作者、時間、內容...等資訊。由上圖可以發現，社團貼文的類名（classname）是 `x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z`，因此 CSS Selector 可以寫成 `.x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z`。

In [20]:
elements = soup.select('.x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z')

# 輸出隨便一篇貼文元素看看
elements[3]

<div class="x1yztbdb x1n2onr6 xh8yej3 x1ja2u2z"><div class="x1n2onr6 x1ja2u2z"><div class=""><div class=""><div aria-describedby=":r3p: :r3q: :r3r: :r3t: :r3s:" aria-labelledby=":r3o:" aria-posinset="3" class="x1a2a7pz" role="article"><div class="x78zum5 xdt5ytf"><div class="x9f619 x1n2onr6 x1ja2u2z"><div class="x78zum5 x1n2onr6 xh8yej3"><div class="x9f619 x1n2onr6 x1ja2u2z x1jx94hy x1qpq9i9 xdney7k xu5ydu1 xt3gfkd xh8yej3 x6ikm8r x10wlt62 xquyuld" style="border-radius: max(0px, min(var(--card-corner-radius), calc((100vw - 4px - 100%) * 9999))) / var(--card-corner-radius);"><div><div></div><div><div class="html-div xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd"><div class="html-div xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd"><div class=""></div><div class="html-div xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd"><div class="html-div xe8uvvx xdj266r x11i5rnm x1mh8g0r x18d9i69 x1cy8zhl x78zum5 x1q0g3np 

### 爬取貼文資訊

#### 01. 貼文作者  

情境：目標元素的 HTML Element 不包含屬性值。
解法：加入目標元素的父級。

![author_position](./image/author_position.png)

可以觀察到「作者」的網頁元素位置在 `span` 標籤下，由於沒有標記任何屬性（class 或 id），直接指定 `span` 的話很容易抓到錯誤的目標。

這時，就需要給 CSS Selectors 更多的線索來找到目標的元素。

以目標元素的位置為中心，往前找到最近的「父級元素」。

> 只要是把目標元素包裹在自己的標籤下，都算是目標元素的父級！
> 當然距離越遠，關係越遠，越容易受到其他節點影響喔

在這個案例中，節點的關係為：

span.xt0psk2（父級 3） > a.x1i10hfl.xjbqb8w...（父級 2） > strong（父級 1） > span（目標）

1. `strong` 標籤不具屬性值，不是最好的選擇。
2. `a` 具有屬性值，但 classname 太多，太過於具體反而不好維護（網站更新後較容易失效）。
3. `span.xt0psk2` classname 不多，且其他地方沒有重複的屬性值，因此偏好加入這個父級到 CSS Selector。

當然，其實使用第二步的 `a` 標籤還是可以達到同樣的效果！不過如果能用更短、更精簡的 CSS Selector，可讀性及效率會較高。

In [21]:
for element in elements:
    try:
        name = element.select('.xt0psk2 span')[0].text
        
        # 👇使用 a 標籤的效果一樣，但 CSS Selector 相比之下長很多
        # name = element.select('.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xt0psk2.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz.xt0b8zv.xzsf02u.x1s688f span')[0].text
        print(name)
    except:
        continue

酒造和食居酒屋
94愛旅遊優惠搶先報
94愛旅遊優惠搶先報
張小冉
江曼寧


#### 02. 貼文按讚數

情境：目標元素的 HTML Element 在其他節點重名了（標籤與 classname 相同）。
解法：加入目標元素的父級。

![like_position](./image/like_position.png)

「貼文讚數」對應的網頁元素是 classname 為 `x1e558r4` 的 `span` 標籤。那麼我們就直接指定 `span.x1e558r4span.x1e558r4` 看看效果如何。

In [23]:
for element in elements:
    try:
        like = element.select('span.x1e558r4')[0].text
        print(element.select('span.x1e558r4')[0]) 
    except:
        continue

<span class="x6s0dn4 x78zum5 x1e558r4" id=":r4e:"><span class="x6zyg47 x1xm1mqw xpn8fn3 xtct9fg x13zp6kq x1mcfq15 xrosliz x1wb7cse x13fuv20 xu3j5b3 x1q0q8m5 x26u7qi xamhcws xol2nv xlxy82 x19p7ews xmix8c7 x139jcc6 x1n2onr6 x1xp8n7a xhtitgo"><span class="x12myldv x1udsgas xrc8dwe xxxhv2y x1rg5ohu xmix8c7 x1xp8n7a"><span class="x4k7w5x x1h91t0o x1h9r5lt x1jfb8zj xv2umb2 x1beo9mf xaigb6o x12ejxvf x3igimt xarpa2k xedcshv x1lytzrv x1t2pt76 x7ja8zs x1qrby5j"><div aria-label="讚：7人" class="x1i10hfl x1qjc9v5 xjbqb8w xjqpnuy xa49m3k xqeqjp1 x2hbi6w x13fuv20 xu3j5b3 x1q0q8m5 x26u7qi x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xdl72j9 x2lah0s xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r x2lwn1j xeuugli xexx8yu x4uap5 x18d9i69 xkhd6sd x1n2onr6 x16tdsg8 x1hl2dhg xggy1nq x1ja2u2z x1t137rt x1o1ewxj x3x9cwd x1e5q0jg x13rtm0m x3nfvp2 x1q0g3np x87ps6o x1lku1pv x1a2a7pz" role="button" tabindex="0"><img class="x16dsc37" height="18" role="presentation" src="data:image/svg+xml,%3Csvg fill='none' xmlns='http

從結果發現抓到的元素不是我們要的。

如上圖，抓到的是其他相同標籤與 classname 的元素，所以我們需要更多關於目標元素的資訊。

與前面的例子相同，在加入父級元素後，指定 CSS Selector 為 `.xt0b8zv.x2bj2ny.xrbpyxo.xl423tq span.x1e558r4`。

In [24]:
for element in elements:
    try:
        like = element.select('.xt0b8zv.x2bj2ny.xrbpyxo.xl423tq span.x1e558r4')[0].text
        print(like)
    except:
        continue

7


這樣便順利抓到我們要的結果！

#### 03. 貼文發佈時間

情境：目標元素的 HTML Element 不包含屬性值。
解法：加入目標元素的父級。

![time_position](./image/time_position.png)

這邊的 CSS Selector 的指定邏輯與「貼文作者」一樣，但周圍父級元素的 classname 都不短，所以就直接挑最近的父級元素來指定。

In [25]:
for element in elements:
    try:
        date_time = element.select('.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xt0psk2.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz.x1heor9g.xt0b8zv.xo1l8bm span')[0].text
        print(date_time)
    except:
        continue

1分鐘
15分鐘
40分鐘
5天
1小時


#### 04. 貼文連結

情境：爬取目標是 `href` 屬性。
解法：使用 `get` 來抓指定屬性的屬性值。

![url_position](./image/url_position.png)

正好 `a` 標籤下的 `href` 屬性便是貼文連結。直接指定該元素的 classname 後，使用 `get('href')`抓出貼文連結。

Facebook 的社團貼文連結結構是 https://www.facebook.com/groups/『社團 id』/posts/『貼文 id』。

從畫面可以看到貼文連結非常長，而我們只需要連結的主要部分即可，所以這邊利用正規表達式，把參數前的部分連結擷取出來即可。

> 實際把長連結放進瀏覽器中，一樣會重定向到「https://www.facebook.com/groups/『社團 id』/posts/『貼文 id』」的連結結構。該參數多半是 FB 內部用來追蹤用戶行為或其他用途，因為不確定參數的用途，保險起見過濾掉較好
> 有了貼文連結後，要補充更多資料就更容易了！例如：爬取留言。

In [26]:
import re

for element in elements:
    try:
        link = element.select('.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xt0psk2.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz.x1heor9g.xt0b8zv.xo1l8bm')[0].get('href')
        # 正規表達式
        base_url = re.match(r"(https://www\.facebook\.com/groups/traveler168/posts/\d+)", link)

        if base_url:
            print(base_url.group(1))
        else:
            print(link)
    except:
        continue

https://www.facebook.com/groups/traveler168/posts/1860775761104466
https://www.facebook.com/groups/traveler168/posts/1860770161105026
https://www.facebook.com/groups/traveler168/posts/1860759267772782
https://www.facebook.com/groups/traveler168/posts/1857260864789289
https://www.facebook.com/groups/traveler168/posts/1860748664440509


#### 04. 貼文內容

情境：欲爬取的目標文字散落在各個網頁節點下。
解法：使用 `get_text` 來抓剛元素下的所有文本。

![](./image/content_position.png)

由上圖可以發現，Facebook 貼文內容文字分散在各個 HTML Element 內，由於每篇文的結構未必相同。因此很難一個個抓出來。

此時，可以使用 `get_text` 的方法來抓出網頁元素下的所有文本，這樣就不用考慮內文的元素有哪些了。

> `select_one` 的方法會回傳第一個符合 CSS Selector 的元素，所以不需要加入 `[0]` 索引出第一筆。
> 換言之，這邊也可以改回使用 `select`！只是一定要所引出第一個元素，否則無法使用 `get_text`

In [27]:
for element in elements:
    try:
        article = element.select_one('.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1lliihq.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.xudqn12.x3x7a5m.x6prxxf.xvq8zen.xo1l8bm.xzsf02u.x1yc453h')
        print(article.get_text())
    except:
        continue

大家一起吃起來
泡湯+美食，繁忙之餘的小確幸


我們把前面的爬蟲程式組合在一起，建構成函式。輸出成 DataFrame 的格式來看一下結果。

In [28]:
import pandas as pd

def getPostData(elements):
    name_list, like_list, date_list, link_list, article_list = [], [], [], [], []
    for element in elements:
        try:
            name = element.select('.xt0psk2 span')[0].text
            name_list.append(name)
        except:
            continue

        try:
            like = element.select('.xt0b8zv.x2bj2ny.xrbpyxo.xl423tq span.x1e558r4')[0].text
            like_list.append(like)
        except:
            like_list.append(0)

        try:
            date_time = element.select('.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xt0psk2.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz.x1heor9g.xt0b8zv.xo1l8bm span')[0].text
            date_list.append(date_time)
        except:
            date_list.append('')

        try:
            link = element.select('.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xt0psk2.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz.x1heor9g.xt0b8zv.xo1l8bm')[0].get('href')
            base_url = re.match(r"(https://www\.facebook\.com/groups/traveler168/posts/\d+)", link)
            if base_url:
                link_list.append(base_url.group(1))
            else:
                link_list.append(link)
        except:
            link_list.append('')

        try:
            article = element.select_one('.x193iq5w.xeuugli.x13faqbe.x1vvkbs.x1xmvt09.x1lliihq.x1s928wv.xhkezso.x1gmr53x.x1cpjm7i.x1fgarty.x1943h6x.xudqn12.x3x7a5m.x6prxxf.xvq8zen.xo1l8bm.xzsf02u.x1yc453h')
            article_list.append(article.get_text())
        except:
            article_list.append('')

    df = pd.DataFrame({
        '作者': name_list,
        '讚數': like_list,
        '時間': date_list,
        '連結': link_list,
        '貼文內容': article_list
    })
    return df

post_df = getPostData(elements)
post_df

,作者,讚數,時間,連結,貼文內容
0,酒造和食居酒屋,0,1分鐘,https://www.facebook.com/groups/traveler168/po...,大家一起吃起來
1,94愛旅遊優惠搶先報,0,15分鐘,https://www.facebook.com/groups/traveler168/po...,
2,94愛旅遊優惠搶先報,0,40分鐘,https://www.facebook.com/groups/traveler168/po...,
3,張小冉,7,5天,https://www.facebook.com/groups/traveler168/po...,泡湯+美食，繁忙之餘的小確幸
4,江曼寧,0,1小時,https://www.facebook.com/groups/traveler168/po...,


### 貼文內文部分隱藏

從輸出的資料表可以發現，有幾筆資料出現「查看更多」的字樣。

![](./image/dataframe_more.png)

實際在網頁上看到的就像這個樣子，必須點擊「查看更多」才有辦法看到完整貼文內容。

所以需要透過一些方式，讓瀏覽器依序點開每一篇的「查看更多」按鈕，藉此獲得完整的資料。

![more_content_sample](./image/more_content_sample.png)

![more_btn_position](./image/more_btn_position.png)

我們抓到「查看更多」按鈕的 classname 後，可以使用 `find_elements` 來指定。但是，實際上這個 classname 在很多地方大量重複，例如：貼文影片、分享的貼文區塊...等（這裡就不列出有出現的位置，感興趣的話可以檢查看看~）。

為了解決這個問題，我們可以在展開全文的過程中，判斷按鈕的類型。利用 `get_attribute('textContent')` 可以抓出該元素的文字內容，如果不是「查看更多」的話，便會跳過而不點擊。

> 並不是標籤名稱為 `Button` 的元素才能夠點擊喔！

另外在最外層加入了 try 和 except 的語法，來進行例外處理。當 try 區段內的程式發生錯誤時，就會執行 except 裡的內容，如果 try 的程式沒有錯誤，就不會執行 except 的內容。

In [29]:
try:
    # 抓取所有「查看更多」按鈕的元素
    target_class = '.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xt0psk2.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz.xt0b8zv.xzsf02u.x1s688f'
    targets = driver.find_elements(By.CSS_SELECTOR, target_class)
    print(f"展開貼文：{len(targets)}")
    
    # 點擊每個按鈕
    for target in targets:
        try:
            # 如果谮內容為「查看更多」才會點擊，其餘跳過
            if "查看更多" in target.get_attribute('textContent'):
                target.click()
                time.sleep(0.3)
        except:
            continue
except Exception as e:
    # 輸出錯誤訊息
    print(e)
    # 略過
    pass

展開貼文：13


此時，我們再爬取一次資料看看

In [30]:
def getElement(driver):
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    elements = soup.select('.x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z')
    return elements

elements = getElement(driver)

post_df = getPostData(elements)
post_df

,作者,讚數,時間,連結,貼文內容
0,酒造和食居酒屋,0,4分鐘,https://www.facebook.com/groups/traveler168/po...,大家一起吃起來
1,94愛旅遊優惠搶先報,0,17分鐘,https://www.facebook.com/groups/traveler168/po...,
2,94愛旅遊優惠搶先報,0,42分鐘,https://www.facebook.com/groups/traveler168/po...,
3,張小冉,7,5天,https://www.facebook.com/groups/traveler168/po...,泡湯+美食，繁忙之餘的小確幸
4,江曼寧,0,1小時,https://www.facebook.com/groups/traveler168/po...,
5,黃檸檬YELLOWLEMON,0,1小時,https://www.facebook.com/groups/traveler168/po...,#洛杉磯 西好萊塢艾迪遜酒店 The West Hollywood EDITION 詳細開箱...
6,陳冰穎,22,1月24日,https://www.facebook.com/groups/traveler168/po...,來推薦一間新開幕的足體養生會館「大都會足體養生會館」按完會讓您想大喊再來一次，每位專業師傅的...


檢查一下結果，看來都抓到完整的文章內容了！

In [31]:
post_df['貼文內容'].values

array(['大家一起吃起來', '', '', '泡湯+美食，繁忙之餘的小確幸', '',
       '#洛杉磯 西好萊塢艾迪遜酒店 The West Hollywood EDITION 詳細開箱 https://chic.tw/s/WMr8h6xEDITION 是萬豪旗下標榜高端、潮流的設計酒店品牌，西好萊塢艾迪遜酒店是美西首間喔。…… 查看更多',
       '來推薦一間新開幕的足體養生會館「大都會足體養生會館」按完會讓您想大喊再來一次，每位專業師傅的手法都招招到位，由淺層再到深層，疏通您所有經絡，維持全身的氣血順暢，極致放鬆指壓按摩不管是經絡或是穴道加壓，緩解身體所有痠痛，都能夠為您解決…… 查看更多'],
      dtype=object)

### 過載貼文內容隱藏

嘗試抓30筆貼文看看結果如何

In [32]:
driver.refresh()

amount = 0
while amount < 30:
    
    counter = 0
    while counter <= 3:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.5)
        counter += 1

    elements = getElement(driver)
    amount = len(elements)

In [33]:
try:
    target_class = '.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xt0psk2.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz.xt0b8zv.xzsf02u.x1s688f'
    targets = driver.find_elements(By.CSS_SELECTOR, target_class)
    print(f"展開貼文：{len(targets)}")
    
    for target in targets:
        try:
            if "查看更多" in target.get_attribute('textContent'):
                target.click()
                time.sleep(0.3)
        except:
            continue
except Exception as e:
    print(e)
    pass

elements = getElement(driver)

post_df = getPostData(elements)
post_df

展開貼文：29


,作者,讚數,時間,連結,貼文內容
0,Kinmen Tour,0,16小時,https://www.facebook.com/groups/traveler168/po...,無論你是尋找冒險、靜謐還是奇幻我們的 #極光團將帶你穿越極地探索大自然的神秘之美準備好啟程了...
1,魚路小棧精緻鍋物,1,23小時,https://www.facebook.com/groups/traveler168/po...,Drama抓媽日常家庭聚餐也可以一人開一鍋！超大包廂還有卡拉OK日式木質風格設計的鍋物店蔬食...
2,跟著領隊玩,5,20小時,https://www.facebook.com/groups/traveler168/po...,東海岸最美的沙灘之一，在岸邊玩水看魚只是基本，其實你還可以浮潛、SUP和獨木舟，用不同的角度...
3,愛樂,4,18小時,https://www.facebook.com/groups/traveler168/po...,2024台北新藝術博覽會 周末歡迎共襄盛舉 倒數計 精采預告 【2024 台北新藝術博覽會‧...
4,跟著領隊玩,3,19小時,https://www.facebook.com/groups/traveler168/po...,到台東池上看金城武樹騎伯朗大道，當然要住在田裡面啦！你有看到我的民宿嗎！就是被大地畫布水田包...
5,啊賀！！現做捲餅舖 - 外星人小舖,1,1天,https://www.facebook.com/groups/traveler168/po...,啊賀！！現做捲餅舖的「加熱即食」冷凍捲餅上市囉！只要5~8分鐘 就能輕鬆解決您的早、午、晚餐...
6,陳怡如,1,1天,https://www.facebook.com/groups/traveler168/po...,熊寶貝多元化派車即日起邀請朋友加入官網即贈$20乘車金！不限次數 可累積！服務範圍：桃園、雙...
7,謝露西,1,20小時,https://www.facebook.com/groups/traveler168/po...,"義大利初遊!!!!超級超級喜歡波隆那這個城市的>""""""""<"
8,林子恩,2,20小時,https://www.facebook.com/groups/traveler168/po...,2024年最夯的必備景 牛奶海登場囉 5-9月宜蘭龜山島牛奶海遊艇團來啦 趕緊私訊小編卡位...
9,金永豐,0,21小時,https://www.facebook.com/groups/traveler168/po...,東京愛玩樂~高雄出發親子同遊 大人小孩都開心頂級夢幻國度-東京迪工尼著名姻綠神社-冰川神社人...


最後輸出的資料居然只有12筆？但是執行 `len(elements)` 會發現實際載入的貼文確實超過30筆

In [34]:
len(elements)

35

檢查網頁元素後發現，前面載入的20多篇貼文，都被加上的 `hidden` 隱藏起來了，所以爬蟲才會抓不到。

像這種以不斷滾動來載入文章內容的網頁，多少都會有這種類似的機制。為了避免過度消耗瀏覽器的記憶體和處理資源，會透過一些方式去降低資料消耗。以 Facebook 的做法，就是將不在可視範圍內（viewport）中的貼文標記為 `hidden`。

![](./image/hidden.png)

既然不能一次載入完爬取，我們可以嘗試一邊滾動頁面，一邊爬取貼文資料。在每一輪滾動完頁面後，就儲存一次當前頁面的貼文資料。

但是如果今天要爬取大量的資料，例如：100筆、1000筆。即便大部分元素的主要內容會被隱藏，但也會有100、1000個沒有意義的網頁節點保留在頁面上，爬蟲執行的時間也會變得非常長。此時，我們可以借助 Javascript 的語法，在不斷載入頁面與爬取的過程中，同時刪除已標記 hidden 的元素

In [35]:
# 腳本的邏輯是只要有抓到子級存在 hidden 的元素，就刪除離子級最近的貼文元素（.x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z，也就是他自己）
js_script =  '''
    let elements = document.querySelectorAll(".x9f619.x1n2onr6.x1ja2u2z.x1s85apg[hidden]");
    for (let element of elements) {
        let parentElement = element.closest(".x1yztbdb.x1n2onr6.xh8yej3.x1ja2u2z");
        parentElement.parentNode.removeChild(parentElement);
    }
'''

執行 Javascript 腳本

In [36]:
driver.execute_script(js_script)

### 完整爬蟲程式

In [39]:
driver.refresh()

In [40]:
result_df = pd.DataFrame()

amount = 0
while amount < 50:
    
    counter = 0
    while counter <= 3:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.5)
        counter += 1

    try:
        target_class = '.x1i10hfl.xjbqb8w.x1ejq31n.xd10rxx.x1sy0etr.x17r0tee.x972fbf.xcfux6l.x1qhh985.xm0m39n.x9f619.x1ypdohk.xt0psk2.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.xexx8yu.x4uap5.x18d9i69.xkhd6sd.x16tdsg8.x1hl2dhg.xggy1nq.x1a2a7pz.xt0b8zv.xzsf02u.x1s688f'
        targets = driver.find_elements(By.CSS_SELECTOR, target_class)
        
        for target in targets:
            try:
                if "查看更多" in target.get_attribute('textContent'):
                    target.click()
                    time.sleep(0.3)
            except:
                continue
    except Exception as e:
        print(e)
        pass

    elements = getElement(driver)
    post_df = getPostData(elements)

    # 每一輪爬取的資料儲存進 result_df（與 result_df 合併）
    result_df = pd.concat([result_df, post_df])

    time.sleep(0.3)

    driver.execute_script(js_script)

    result_df['貼文內容'] = result_df['貼文內容'].astype(str)
    amount = len(result_df[~result_df['貼文內容'].str.contains('查看更多')]['連結'].unique())
    print(f"爬取進度: {amount} 筆")

爬取進度: 8 筆
爬取進度: 18 筆
爬取進度: 27 筆
爬取進度: 39 筆
爬取進度: 51 筆


In [43]:
driver.close()

因為是一邊滾動一邊抓資料，貼文有可能會重複抓取，所以我們依據「連結」來篩選出不重複的資料。

不過，為什麼明明爬蟲時，展開每篇貼文的隱藏內容，還會抓到沒展開的貼文？

仔細觀察展開完整貼文時 Webdriver 的變化，可以發現 Webdriver 會將畫面移動至該篇文出現在可視範圍內。假設現在有一篇需要展開的貼文，恰好位在網頁的最後幾個元素上，那麼當 Webdriver 移動到該位置（頁面底部）並進行點擊時，便會自動刷出新的貼文。若新載入的貼文正好是需要展開的貼文，那麼就會抓到未展開的貼文。

畢竟需要點擊的貼文，在貼文自動刷新前就決定好了~

不過這問題影響不大，下一輪滾動完後，這一篇文就會進到需要被點擊的貼文清單了。

In [41]:
# 篩選掉「貼文內容」有出現「查看更多」字樣的資料
dataset = result_df[~result_df['貼文內容'].str.contains('查看更多')]

# drop_duplicates 會依據「連結」篩除重覆的，只保留第一筆出現的資料
dataset = dataset.drop_duplicates(subset = '連結').reset_index(drop = True)

dataset

,作者,讚數,時間,連結,貼文內容
0,酒造和食居酒屋,0,6分鐘,https://www.facebook.com/groups/traveler168/po...,大家一起吃起來
1,94愛旅遊優惠搶先報,0,19分鐘,https://www.facebook.com/groups/traveler168/po...,
2,94愛旅遊優惠搶先報,0,44分鐘,https://www.facebook.com/groups/traveler168/po...,
3,張小冉,7,5天,https://www.facebook.com/groups/traveler168/po...,泡湯+美食，繁忙之餘的小確幸
4,江曼寧,0,1小時,https://www.facebook.com/groups/traveler168/po...,
5,黃檸檬YELLOWLEMON,0,1小時,https://www.facebook.com/groups/traveler168/po...,#洛杉磯 西好萊塢艾迪遜酒店 The West Hollywood EDITION 詳細開箱...
6,陳冰穎,22,1月24日,https://www.facebook.com/groups/traveler168/po...,來推薦一間新開幕的足體養生會館「大都會足體養生會館」按完會讓您想大喊再來一次，每位專業師傅的...
7,跟著領隊玩,2,3小時,https://www.facebook.com/groups/traveler168/po...,Google高達4.3顆星！新北CP值爆表的生魚片，300元就有將近30片的鮭魚、紅甘、旗魚...
8,衙門打工仔的日常,0,5小時,https://www.facebook.com/groups/traveler168/po...,更新澳洲自由行8天7夜的第3天到現在還是非常想念墨爾本雖然墨爾本沒有一個顯著的地標不過天際線...
9,Chen Li,0,5小時,https://www.facebook.com/groups/traveler168/po...,邀約你風和日麗的好天氣時踏青爬山到汐碇路(或白雲路、勤進路、秀峰路、大尖山步道)踏雪尋幽，數...


將資料儲存成 csv 格式

In [42]:
result_df.to_csv('./data/fb_post_01.csv', index = False)